In [8]:
import dash
from dash import dcc, html, callback, Output, Input, State
import base64
import io
import polars as pl
from natsort import natsorted

app = dash.Dash(__name__)

model = None
states = None
options = {}
multi_options = {}
static_options = {}

# Placeholder for storing dataset options
app.layout = html.Div([
    dcc.Upload(id='upload-data', children=html.Button('Upload Data')),
    dcc.Store(id='multi-options'),
    dcc.Store(id='static-options'),
    dcc.Store(id='clustermap'),
    html.Div(id='static-options-container'),
    html.Div(id='slider-container')  # This will hold the sliders
])

@callback(
    [Output('multi-options', 'data'),
     Output('static-options', 'data'),
     Output('clustermap', 'data')],
    Input('upload-data', 'contents'),
    State('upload-data', 'filename')
)
def load_data(contents, filename):
    if contents is None:
        return {}, {}

    content_type, content_string = contents.split(',')
    decoded = base64.b64decode(content_string)
    
    # Use io.BytesIO to create a file-like object from the decoded content
    model = pl.read_csv(io.BytesIO(decoded), separator="\t")
    data_cols = ["file1", "file2", "scc"]

    options = {col: natsorted(model[col].unique())
               for col in model.columns
               if col not in data_cols}
    
    states = model.select(options.keys()).unique()
    
    multi_options = {col: options[col]
                     for col in options
                     if len(options[col]) > 1}
    static_options = {col: options[col]
                     for col in options
                     if len(options[col]) == 1}

    return multi_options, static_options

@callback(
    Output('slider-container', 'children'),
    Input('multi-options', 'data')
)
def update_multi_options(new_multi_options):
    if not new_multi_options:
        return []

    slider_elements = []
    for option_name, option_settings in new_multi_options.items():
        slider_elements.append(
            html.Div([
                dcc.Slider(id = option_name,
                           min = 0,
                           max = len(option_settings)-1,
                           step=None,
                           value = 0,
                           marks = {str(i): str(setting) for i, setting in enumerate(option_settings)})
            ], style={'margin': '10px 0'})
        )

    return slider_elements

@callback(
    Output('static-options-container', 'children'),
    Input('static-options', 'data')
)
def update_static_options(new_static_options):
    if not new_static_options:
        return []

    p_elements = []
    for option_name, option_settings in new_static_options.items():
        p_elements.append(
            html.Div([html.P(f"{option_name}: {option_settings[0]}")])
        )

    return p_elements

@callback(
    Input('')
)

if __name__ == '__main__':
    app.run_server(debug=True)

@callback(

)
